### Imports

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import seaborn as sns

sns.set_theme(style='ticks')
sns.set_context('paper')

# from pycaret.regression import *

In [2]:
# plot two features against each other
def plot_jointplot(data, x, y):
    r, p = sp.stats.pearsonr(data[x], data[y])

    jgrid = sns.jointplot(data=data, x=x, y=y, kind='reg')
    jgrid.ax_joint.text(
        1, 1, 
        f"r={r:.2f}, p={p:.2e}",
        horizontalalignment='right',
        transform = jgrid.ax_joint.transAxes
    )

    jgrid.figure.show()

In [3]:
data = gpd.read_file('../GENERATED-DATA/data_by_zone.geojson')
data.shape

(342, 78)

### Plots

In [ ]:
data.plot(
    column='crimes',
    legend=True,
    linewidth=0.1,
    edgecolor='0.8',
    figsize=(10, 10),
    cmap='viridis_r',
    norm=LogNorm(vmin=1, vmax=data['crimes'].max()),
)

plt.xticks([])
plt.yticks([])

plt.show()


In [ ]:
data.plot(
    column='V001_BASICO',
    legend=True,
    linewidth=0.8,
    edgecolor='0.9',
    figsize=(10, 10),
    cmap='viridis_r',
)

plt.xticks([])
plt.yticks([])

plt.show()


### Exploring the connection between crime and other variables

#### CENSUS Variables

In [ ]:
df = data[[
    'Area_ha_2', 'V001_ENTORNO01', 'V002_ENTORNO01', 'V003_ENTORNO01',
    'V004_ENTORNO01', 'V001_DOMICILIORENDA', 'V001_BASICO', 'V002_BASICO',
    'V001_DOMICILIO02', 'V002_DOMICILIO02', 'V001_DOMICILIO01',
    'V002_DOMICILIO01', 'V001_PESSOA01', 'V086_PESSOA02', 'V001_PESSOA03',
    'V002_PESSOA03', 'V003_PESSOA03', 'V004_PESSOA03', 'V005_PESSOA03',
    'V006_PESSOA03', 'V001_PESSOA12', 'V001_PESSOA11', 'V001_RESPONSAVEL01',
    'V001_RESPONSAVEL02', 'V002_DOMICILIORENDA', 'V003_DOMICILIORENDA',
    'V004_DOMICILIORENDA', 'V003_BASICO', 'V005_BASICO', 'V007_BASICO',
    'V009_BASICO', 'V011_BASICO', 'crimes'
]]

df.shape

In [ ]:
ax = sns.histplot(data=df, x='V001_BASICO', bins=20, kde=True)

In [ ]:
plot_jointplot(df, 'crimes', 'V001_BASICO')

In [ ]:
sns.boxplot(data=df, x='crimes')

In [20]:
# remove outliers based on the IQR
Q1 = df['crimes'].quantile(0.25)
Q3 = df['crimes'].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df['crimes'] < (Q1 - 1.5 * IQR)) | (df['crimes'] > (Q3 + 1.5 * IQR)))]

In [ ]:
plot_jointplot(df, 'crimes', 'V001_BASICO')

In [ ]:
# calcular pearson r for all features
corr_matrix = df.corr().abs()

# show as dataframe
corr_matrix['crimes'].sort_values(ascending=False).to_frame()[:10]

In [ ]:
# it seems to be a lot of multicollinearity with V001_BASICO
# Domicílios particulares permanentes ou pessoas responsáveis por domicílios particulares permanentes

corr_matrix['V001_BASICO'].sort_values(ascending=False).to_frame()[:10]

In [24]:
df = df.loc[:, (df.columns == 'V001_BASICO') | (corr_matrix['V001_BASICO'] < 0.8)]

In [ ]:
# calcular pearson r for all features
corr_matrix = df.corr().abs()

# show as dataframe
corr_matrix['crimes'].sort_values(ascending=False).to_frame()[:10]

In [ ]:
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

cols = (corr_matrix.mask(mask) > .8).any()
sns.heatmap(corr_matrix.loc[cols, cols], cmap='viridis')

In [ ]:
# calcular pearson r for all features
corr_matrix = df.corr().abs()

# show as dataframe
corr_matrix['V005_BASICO'].sort_values(ascending=False).to_frame()[:10]

In [28]:
df = df.loc[:, (df.columns == 'V005_BASICO') | (corr_matrix['V005_BASICO'] < 0.8)]

In [ ]:
df.columns

### Regression

In [5]:
import tpot 
from deap import creator

def tpot_models_dataframe(tpot_model):
    scores = {}
    for model, model_info in tpot_model.evaluated_individuals_.items():
        cv_score = model_info.get('internal_cv_score')
        scores[model] = cv_score

    scores = (
        pd.DataFrame.from_dict(
            scores,
            orient='index',
            columns=['cv_score']
        )
        .sort_values(
            by='cv_score',
            ascending=False
        )
        .reset_index(names='model')
    )

    return scores

def tpot_pipeline_from_string(tpot_model, pipeline_string):
    deap_pipeline = creator.Individual.from_string(pipeline_string, tpot_model._pset)
    sklearn_pipeline = tpot_model._toolbox.compile(expr=deap_pipeline)
    return sklearn_pipeline

In [53]:
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyRegressor

def compare_metric_with_dummy(model, score, X, y, cv):
    dummy = DummyRegressor(strategy='mean')

    cross_val_dummy = cross_val_score(
        dummy,
        X, y,
        cv=cv,
        scoring=score,
    )

    cross_val_model = cross_val_score(
        model,
        X, y,
        cv=cv,
        scoring=score,
    )

    df = pd.DataFrame()
    df['dummy_scores'] = cross_val_dummy
    df['model_scores'] = cross_val_model
    return df.sort_values(by='model_scores', ascending=False)


In [7]:
features_to_drop = [
    'NumeroZona',
    'NomeZona',
    'NumeroMuni',
    'NomeMunici',
    'NumDistrit',
    'NomeDistri',
    'geometry',
]

features = [feature for feature in data.columns.to_list() if feature not in features_to_drop]
data = data[features]
data.dropna(inplace=True)

In [8]:
data.to_csv('../GENERATED-DATA/data_by_zone.csv', index=False)

In [45]:
X = data.drop(columns='crimes')
y = data['crimes']

### TPOT

In [10]:
from tpot import TPOTRegressor
from sklearn.model_selection import RepeatedKFold

cv = RepeatedKFold(
    n_splits=10,
    n_repeats=3,
    random_state=123,
)

tpot_model = TPOTRegressor(
    max_time_mins=60*12,
    verbosity=3,
    random_state=123,
    cv=cv,
    scoring='r2',
    n_jobs=-1,
    warm_start=True,
    memory='auto',
    template='Selector-Transformer-Regressor',
)

tpot_model.fit(X, y)

30 operators have been imported by TPOT.
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required by RobustScaler..
_pre_test decorator: _random_mutation_operator: num_test=1 FeatureAgglomeration.__init__() got an unexpected keyword argument 'affinity'.
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required by FastICA..
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False.
_pre_test decorator: _random_mutation_operator: num_test=0 LassoLarsCV.__init__() got an unexpected keyword argument 'normalize'.
_pre_test decorator: _random_mutation_operator: num_test=1 FeatureAgglomeration.__init__() got an unexpected keyword argument 'affi

TPOTRegressor(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=123),
              max_time_mins=720, memory='auto', n_jobs=-1, random_state=123,
              scoring='r2', template='Selector-Transformer-Regressor',
              verbosity=3, warm_start=True)

In [66]:
models = tpot_models_dataframe(tpot_model)
#models.to_csv('..../tpot_models.csv', index=False)
models[:20]

,model,cv_score
0,GradientBoostingRegressor(OneHotEncoder(Varian...,0.572451
1,GradientBoostingRegressor(OneHotEncoder(Varian...,0.572451
2,GradientBoostingRegressor(OneHotEncoder(Varian...,0.572451
3,GradientBoostingRegressor(OneHotEncoder(Varian...,0.572451
4,GradientBoostingRegressor(OneHotEncoder(Varian...,0.572451
5,GradientBoostingRegressor(RobustScaler(Varianc...,0.571964
6,GradientBoostingRegressor(OneHotEncoder(Varian...,0.570530
7,GradientBoostingRegressor(OneHotEncoder(Varian...,0.570530
8,GradientBoostingRegressor(OneHotEncoder(Varian...,0.570530
9,GradientBoostingRegressor(OneHotEncoder(Varian...,0.570530


In [19]:
tpot_model.export('../GENERATED-DATA/tpot_pipeline.py')

In [12]:
top_model = tpot_pipeline_from_string(tpot_model, models.model[0])
top_model

Pipeline(steps=[('variancethreshold', VarianceThreshold(threshold=0.005)),
                ('onehotencoder',
                 OneHotEncoder(minimum_fraction=0.25, sparse=False)),
                ('gradientboostingregressor',
                 GradientBoostingRegressor(alpha=0.95, loss='huber',
                                           max_depth=4,
                                           max_features=0.7000000000000001,
                                           min_samples_leaf=4,
                                           min_samples_split=9,
                                           random_state=123,
                                           subsample=0.9000000000000001))])

### GradientBoosting

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingRegressor

vt = VarianceThreshold(threshold=0.005)

gdb = GradientBoostingRegressor(
    alpha=0.95, loss='huber', max_depth=4,
    max_features=0.7, min_samples_leaf=4,
    min_samples_split=9, random_state=123,
    subsample=0.9
)

score = 'neg_mean_absolute_error'
df = compare_metric_with_dummy(gdb, score, vt.fit_transform(X), y, cv)
df.abs().describe()

### XGBoost

In [69]:
from xgboost import XGBRegressor

xgb = XGBRegressor(
    eval_metric='mae',
    n_estimators=8000,
    learning_rate=0.001,
)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)


from sklearn.metrics import mean_absolute_error

y_pred = xgb.predict(X_test)
mean_absolute_error(y_test, y_pred)


[0]	validation_0-mae:692.22454
[100]	validation_0-mae:658.31404
[200]	validation_0-mae:629.16927
[300]	validation_0-mae:596.23414
[400]	validation_0-mae:586.08428
[500]	validation_0-mae:576.62770
[600]	validation_0-mae:567.08581
[700]	validation_0-mae:556.82183
[800]	validation_0-mae:542.41484
[900]	validation_0-mae:528.01215
[1000]	validation_0-mae:515.60835
[1100]	validation_0-mae:504.99191
[1200]	validation_0-mae:496.71221
[1300]	validation_0-mae:491.43247
[1400]	validation_0-mae:489.48956
[1500]	validation_0-mae:488.30415
[1600]	validation_0-mae:486.12718
[1700]	validation_0-mae:482.70212
[1800]	validation_0-mae:478.66841
[1900]	validation_0-mae:475.90683
[2000]	validation_0-mae:474.25970
[2100]	validation_0-mae:472.85148
[2200]	validation_0-mae:471.64724
[2300]	validation_0-mae:470.48991
[2400]	validation_0-mae:470.59322
[2500]	validation_0-mae:469.91419
[2600]	validation_0-mae:468.84747
[2700]	validation_0-mae:467.78297
[2800]	validation_0-mae:467.27985
[2900]	validation_0-mae:46

np.float64(450.33690284280215)

In [70]:
score = 'neg_mean_absolute_error'
df = compare_metric_with_dummy(xgb, score, X, y, cv)
df.abs().describe()

np.float64(-498.81792613826747)

### Add Spacial Lag

In [ ]:
from pysal.lib import weights

knn = weights.KNN.from_dataframe(data, k=5) # k=5 -> 50%
wq  = weights.contiguity.Rook.from_dataframe(data)
w_kernel = weights.distance.Kernel.from_dataframe(data)

In [ ]:
f, axs = plt.subplots(figsize=(10, 10))

ax = data.plot(
    edgecolor="k", facecolor="w", ax=axs
)

knn.plot(
    data,
    ax=axs,
    edge_kws=dict(color="r", linestyle=":", linewidth=1),
    node_kws=dict(marker="")
)

axs.axis([-46.6, -46.8, -23.7, -23.6]);
axs.set_axis_off()

In [ ]:
gdf_lag = (
    data[features]
    .apply(
        lambda y: weights.spatial_lag.lag_spatial(wq, y)
    )
    .rename(
        columns=lambda c: "LAG_" + c
    )
)

In [ ]:
gdf_lag.shape

In [ ]:
gdf_lag.isna().sum()

In [ ]:
# data = data[features + ['crimes']].join(gdf_lag)
# data.columns